In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import setuptools

# Keras is a high-level neural network Python API
# Runs on top of lower-level libraries like TensorFlow
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

In [2]:
class MLClass:

    def __init__(self, data=None, predictorVar=None, targetCol=None,
                xTrain=None, xTest=None, yTest=None, yTrain=None):
        self.data = data
        self.predictorVar = predictorVar
        self.targetCol = targetCol
        self.xTrain = xTrain
        self.xTest= xTest
        self.yTest= yTest
        self.yTrain = yTrain

    
    def exploringDataset(self):      
        try:
            print("The shape of the data is: ", self.data.shape)
            print("The dataset has {} columns".format(self.data.columns))
            print("The name of the columns are: {}\n".foramt([x for x in self.data.columns]))

        except Exception as es:
            print("Data type is None, load the dataset first")


    def cleanData(self):
        playType = ['Run','Pass',"Field Goal",'Punt']
        self.data = self.data[self.data["PlayType"].isin(playType)]
        self.data = self.data.dropna(subset=["PlayType", "down"])
        self.data["FieldGoalDistance"] = self.data["FieldGoalDistance"].fillna(0)

        predictorCols = ["GameID", "Drive", "qtr", "down", "time", "yrdline100", "ydstogo", "GoalToGo", "PosTeamScore", "DefTeamScore",\
                 "FieldGoalDistance", "ScoreDiff"]
        self.__setPredictorVar(predictorCols)
        target = "PlayType"
        self.__setTargetCol(target)


    def __setPredictorVar(self, predictorCols):
        df = self.data[predictorCols]
        df["time"] = df["time"].apply(lambda x: int(x.replace(':','')))
        scaler = StandardScaler().set_output(transform='pandas')
        feature_mat = scaler.fit_transform(df)
        self.predictorVar = feature_mat

    def __setTargetCol(self, targetColumn):
        target = self.data[targetColumn]
        target_col = target.replace({'Pass': 0, 'Run': 1, 'Punt':2, "Field Goal":3})
        self.targetCol = target_col

    def getPredictorVar(self):
        return self.predictorVar

    def getTargetCol(self):
        return self.targetCol

    def getTrain(self):
        try:
            return {"X-Train": self.xTrain,
                     "Y-Train": self.yTrain}
        except Exception as es:
            print("The Training dataset has not been created yet")

    def getTest(self):
        try:
            return{"X-Test": self.xTest,
                   "Y-Test": self.yTest}
        except Exception as es:
            print("The Testing dataset has not been created yet")
            
    def loadDataset(self, filePath):
        self.data = pd.read_csv(filePath)
        return self.data

    def trainTestDataset(self, testSize=0.10, randomState=24):
        self.xTrain, self.xTest, self.yTrain, self.yTest = \
        train_test_split(self.predictorVar, self.targetCol, test_size=testSize, random_state=randomState)

        print("X_train:", self.xTrain.shape); print("X_test:", self.xTest.shape); print("y_train:", self.yTrain.shape); print("y_test:", self.yTest.shape)


In [3]:
class CNNClass(MLClass):

    def __init__(self, model=None, data=None, predictorVar=None, targetCol=None, 
                 xTrain=None, xTest=None, yTest=None, yTrain=None):
        
        self.model = model

        super().__init__(data, predictorVar, targetCol,
                xTrain, xTest, yTest, yTrain)
        
        
    def createCnnModel(self, featuresNum=None, outputDim=None):
        self.model = Sequential()
        self.model.add(Dense(128, activation='relu', input_dim=featuresNum)) 
        self.model.add(Dense(64, activation='relu')) 
        self.model.add(Dense(32, activation='relu'))
        self.model.add(Dense(16, activation='relu'))
        self.model.add(Dense(outputDim))

        self.__compileModel()
        self.__getModelSummary()


    def __compileModel(self):
        self.model.compile(optimizer='adam',
                           loss='mean_squared_error',
                           metrics=['accuracy'])

    def __getModelSummary(self):
        self.model.summary()

    
    def trainModel(self, numEpochs=1):
        train = self.getTrain()
        self.model.fit(train["X-Train"], train["Y-Train"], epochs= numEpochs)

    
    def trainDataEvaluate(self):
        train = self.getTrain()
        loss, accuracy = self.model.evaluate(train["X-Train"],
                                             train["Y-Train"])

        print(f"Training Loss: {loss}")
        print(f"Training Accuracy: {accuracy}")

    def testDataEvaluate(self):
        test = self.getTest()
        loss, accuracy = self.model.evaluate(test["X-Test"],
                                             test["Y-Test"])
        
        print(f"Testing Loss: {loss}")
        print(f"Testing Accuracy: {accuracy}")

    def performanceMeasure(self):
        train = self.getTrain()
        test = self.getTest()

        x_train = train["X-Train"]
        y_train = train["Y-Train"]
        x_test = test["X-Test"]
        y_test = test["Y-Test"]

        history = self.model.fit(x_train, y_train, validation_data=(x_test, y_test))
        history_dict = history.history

        plt.plot(history_dict['accuracy'], label='accuracy')
        plt.plot(history_dict['val_accuracy'], label='val_accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.ylim([0, 1])
        plt.legend(loc='lower right')

#Testing

In [4]:
cnnModel = CNNClass()
data = cnnModel.loadDataset('/Users/gauravkharel/Documents/BoozAllen/Data Science/Capstone/NFLPlaybyPlay2015.csv')
cnnModel.cleanData()

/var/folders/q_/jngx2c4j3d57vcn5b3qsyqcw0000gn/T/ipykernel_27523/2584321588.py:70: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  self.data = pd.read_csv(filePath)
/var/folders/q_/jngx2c4j3d57vcn5b3qsyqcw0000gn/T/ipykernel_27523/2584321588.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["time"] = df["time"].apply(lambda x: int(x.replace(':','')))
/var/folders/q_/jngx2c4j3d57vcn5b3qsyqcw0000gn/T/ipykernel_27523/2584321588.py:46: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downc

In [5]:
cnnModel.trainTestDataset()
cnnModel.createCnnModel(12, 5)

X_train: (31279, 12)
X_test: (3476, 12)
y_train: (31279,)
y_test: (3476,)


/Users/gauravkharel/Documents/BoozAllen/Data Science/Capstone/sklearn-env/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 5)              │            85 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,613 (49.27 KB)

 Trainable params: 12,613 (49.27 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
cnnModel.trainModel(numEpochs=10)

Epoch 1/10
978/978 ━━━━━━━━━━━━━━━━━━━━ 1s 312us/step - accuracy: 0.1342 - loss: 0.3370
Epoch 2/10
978/978 ━━━━━━━━━━━━━━━━━━━━ 0s 305us/step - accuracy: 0.1776 - loss: 0.2004
Epoch 3/10
978/978 ━━━━━━━━━━━━━━━━━━━━ 0s 309us/step - accuracy: 0.1576 - loss: 0.1977
Epoch 4/10
978/978 ━━━━━━━━━━━━━━━━━━━━ 0s 299us/step - accuracy: 0.1596 - loss: 0.1954
Epoch 5/10
978/978 ━━━━━━━━━━━━━━━━━━━━ 0s 302us/step - accuracy: 0.1538 - loss: 0.1907
Epoch 6/10
978/978 ━━━━━━━━━━━━━━━━━━━━ 0s 304us/step - accuracy: 0.1529 - loss: 0.1890
Epoch 7/10
978/978 ━━━━━━━━━━━━━━━━━━━━ 0s 302us/step - accuracy: 0.1535 - loss: 0.1888
Epoch 8/10
978/978 ━━━━━━━━━━━━━━━━━━━━ 0s 301us/step - accuracy: 0.1529 - loss: 0.1876
Epoch 9/10
978/978 ━━━━━━━━━━━━━━━━━━━━ 0s 302us/step - accuracy: 0.1598 - loss: 0.1856
Epoch 10/10
978/978 ━━━━━━━━━━━━━━━━━━━━ 0s 303us/step - accuracy: 0.1537 - loss: 0.1874


In [7]:
cnnModel.performanceMeasure()

978/978 ━━━━━━━━━━━━━━━━━━━━ 0s 405us/step - accuracy: 0.1529 - loss: 0.1861 - val_accuracy: 0.1887 - val_loss: 0.1881


TypeError: 'History' object is not subscriptable